In [ ]:
import sys
import os

sys.path.append('F:/ai-ml/itapia/backend/evo_worker')
sys.path.append('F:/ai-ml/itapia/backend/shared')

In [ ]:
os.environ['POSTGRES_HOST'] = 'localhost'
os.environ['REDIS_HOST'] = 'localhost'

In [ ]:
from app.dependencies import get_backtest_context_manager, create_dependencies
create_dependencies()

In [ ]:
context_mng = get_backtest_context_manager()

In [ ]:
context_mng.init_ready_contexts()

In [ ]:
import nest_asyncio
nest_asyncio.apply()
await context_mng.get_context('AAPL').load_data_into_memory()
await context_mng.get_context('NVDA').load_data_into_memory()
await context_mng.get_context('O').load_data_into_memory()

In [ ]:
contexts = context_mng.get_ready_contexts()

In [ ]:
from app.backtest.evaluator import SingleContextFitnessEvaluator, MultiContextFitnessEvaluator
from app.backtest.action import ShortSwingActionMapper

In [ ]:
action_mapper = ShortSwingActionMapper()

In [ ]:
from itapia_common.dblib.services import RuleService
from itapia_common.dblib.session import get_rdbms_session

db = next(get_rdbms_session())
rule_service = RuleService(db)

In [ ]:
rules = rule_service.get_all_active_rules()

In [ ]:
test_rule_ent = rules[2]

In [ ]:
from itapia_common.rules.rule import Rule

test_rule = Rule.from_entity(test_rule_ent)

In [ ]:
evaluator0 = SingleContextFitnessEvaluator(contexts[0], action_mapper=action_mapper)
evaluator1 = SingleContextFitnessEvaluator(contexts[1], action_mapper=action_mapper)
evaluator2 = SingleContextFitnessEvaluator(contexts[2], action_mapper=action_mapper)

In [ ]:
evaluator0.evaluate(test_rule)

In [ ]:
evaluator1.evaluate(test_rule)

In [ ]:
evaluator2.evaluate(test_rule)

In [ ]:
multi_ctx_evaluator = MultiContextFitnessEvaluator(contexts, action_mapper)

In [ ]:
multi_ctx_evaluator.evaluate(test_rule, 'median')